### Lab_3

В файле sim_lib.py лежит функция simulation которая производит симуляцию жизненного цикла баннеров(Mortal bandits), на некотором абстрактном траффике. Ей на вход подается policy и n кол-во итераций. Policy это функция со следующей спецификацией:

- на вход получает лог исторических наблюдений. Это пандас датафрейм, где каждая строка соответствует статистике по одному баннеру, а индекс датафрейма, это номер этого баннера.
- на выходе выдает индекс в датафрейме отвечающий баннеру, который надо выбрать в текущей итерации.

В результате работы simulation возвращает кумулятивны regret данного полиси при симуляции за n шагов. Пример использования симулятора для e-greedy бандита находится в ноутбуке task3_example.ipynb.

Task:
- Написать свое policy, функцию, которая работает согласно спецификации выше. Важно, чтобы это была не e-greedy, а иная полиси(UCB, Thompson sampling или что-то другое)
- Прогнать свое полиси через simulation при n=200k и фиксированном сиде из task3_example.
- Оптимизировать свое policy так, чтобы общий регрет был ниже того, который выдает полиси из task3_example.
- Затюнить в своем полиси баланс exploration/exploitation так, чтобы минимизировать регрет при данном сиде.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import numpy as np
import pandas as pd

from scipy.stats import beta, uniform, bernoulli, expon
from typing import Callable

ALPHA = 1
BETA = 20
MU = 10 ** 4
MONITORING_FREQ = 10 ** 4
MAX_RANDOM = 1111111


def generate_new_banner(n, a=ALPHA, b=BETA, mu=MU, random_state=None):
    if random_state:
        random_state += 1
    p = beta.rvs(a, b, size=n, random_state=random_state)
    lifetimes = expon.rvs(scale=mu, size=n, random_state=random_state)

    return p, lifetimes


def simulation(policy: Callable, n=10 ** 6, initial_banners=9, seed=None):
    state = pd.DataFrame(np.zeros((initial_banners, 4)), columns=['impressions', 'clicks', 'lifetime', 'p'])
    state['p'], state['lifetime'] = generate_new_banner(initial_banners)
    regret = 0
    max_index = initial_banners
    borning_rate = initial_banners*(1-np.exp(-1/MU))
    random_state = seed

    for i in range(n):
        if uniform.rvs(random_state=random_state) < borning_rate or state.shape[0] < 2:
            p, lifetime = generate_new_banner(1, random_state=random_state)
            new_banner = pd.DataFrame({'impressions': 0, 'clicks': 0, 'lifetime': lifetime, 'p': p}, index=[max_index])
            state = pd.concat([state, new_banner], verify_integrity=True)
            max_index += 1

        index = policy(state[['impressions', 'clicks']].copy())

        assert index in state.index, 'Error, wrong action number'

        p = state.loc[index, 'p']
        reward = bernoulli.rvs(p)
        state.loc[index, 'impressions'] += 1
        state.loc[index, 'clicks'] += reward
        regret = regret + max(state['p']) - p

        state['lifetime'] = state['lifetime'] - 1
        state = state[state['lifetime'] > 0]
        if random_state:
            random_state = 7*random_state % MAX_RANDOM

        if not i % MONITORING_FREQ:
            print('{} impressions have been simulated'.format(i + 1))

    return {'regret': regret, 'rounds': n, 'total_banners': max_index, 'history': state}

In [6]:
import numpy as np
import pandas as pd
import time
from functools import partial

from scipy.stats import randint, uniform

pd.options.mode.chained_assignment = None

In [7]:
def eps_greedy(history: pd.DataFrame, eps: float):
    if uniform.rvs() < eps:
        n = history.shape[0]
        return history.index[randint.rvs(0, n)]

    ctr = history['clicks'] / (history['impressions'] + 10)
    n = np.argmax(ctr)
    return history.index[n]

policy = partial(eps_greedy, eps=0.06)

In [7]:
# seed for homework
seed = 18475
np.random.seed(seed=seed)

start = time.time()
output = simulation(policy, n=200000, seed=seed)
end = time.time()
end - start

1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impressions have been simulated
70001 impressions have been simulated
80001 impressions have been simulated
90001 impressions have been simulated
100001 impressions have been simulated
110001 impressions have been simulated
120001 impressions have been simulated
130001 impressions have been simulated
140001 impressions have been simulated
150001 impressions have been simulated
160001 impressions have been simulated
170001 impressions have been simulated
180001 impressions have been simulated
190001 impressions have been simulated


546.4438161849976

In [8]:
# baseline regret
output['regret'], output['regret']/output['rounds'],  output['total_banners']

(1540.7609683932544, 0.007703804841966272, 184)

In [9]:
output['history']

,impressions,clicks,lifetime,p
153,18970.0,4199.0,18003.025431,0.220134
162,228.0,26.0,1537.166719,0.113780
172,163.0,32.0,19648.592394,0.219968
173,170.0,18.0,12771.474990,0.122694
180,26.0,3.0,4655.819793,0.020061
182,6.0,0.0,889.624649,0.004621
183,1.0,0.0,15187.163761,0.073886


###  Алгоритм верхнего доверительного интервала :

In [8]:
class Upper_Confidence_Bound():
    def __init__(self, C):
        self.c = C
        self.eps = 1e-4
        self.n = 1
    def __call__(self, history):
        explot = history['clicks'] / (history['impressions'] + self.eps)
        explore = np.sqrt(np.log(self.n) / (history['impressions'] + self.eps))
        index_upper_bound = np.argmax(explot + self.c * explore)
        self.n += 1
        return history.index[index_upper_bound]

In [9]:
def find_min_regret(some_c, min_c=1, min_regret=7035):
    for c in some_c:
        policy_new = Upper_Confidence_Bound(c)
        start = time.time()
        output_new = simulation(policy_new, n=200000, seed=seed)
        end = time.time()
        print(f'c = {c}\nTime: {end - start}\n')
        # new regret
        curr_regret = output_new["regret"]
        print(f'New results: {curr_regret}, {output_new["regret"]/output_new["rounds"]},  {output_new["total_banners"]}\n')
        if curr_regret < min_regret:
          min_c = c
          min_regret=curr_regret
    print(f'Minimum regret = { min_regret} with c={min_c}')
    return min_c, min_regret

In [29]:
# seed for homework
seed = 18475
np.random.seed(seed=seed)



some_c = [1, 0.1, 0.01, 0.001, 0.0001]

def find_min_regret(some_c, min_c=1, min_regret=7035):
    for c in some_c:
        policy_new = Upper_Confidence_Bound(c)
        start = time.time()
        output_new = simulation(policy_new, n=200000, seed=seed)
        end = time.time()
        print(f'c = {c}\nTime: {end - start}\n')
        # new regret
        curr_regret = output_new["regret"]
        print(f'New results: {curr_regret}, {output_new["regret"]/output_new["rounds"]},  {output_new["total_banners"]}\n')
        if curr_regret < min_regret:
          min_c = c
          min_regret=curr_regret
    print(f'Minimum regret = { min_regret} with c={min_c}')
    return min_c, min_regret

1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impressions have been simulated
70001 impressions have been simulated
80001 impressions have been simulated
90001 impressions have been simulated
100001 impressions have been simulated
110001 impressions have been simulated
120001 impressions have been simulated
130001 impressions have been simulated
140001 impressions have been simulated
150001 impressions have been simulated
160001 impressions have been simulated
170001 impressions have been simulated
180001 impressions have been simulated
190001 impressions have been simulated
c = 1
Time: 603.648143529892

New results: 7035.399109538254, 0.03517699554769127,  184

1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulat

Минимальное значение regret было достигнуто при с=0.1 и составило 2098. Попробуем уменьшить это значение еще:

In [12]:
seed = 18475
np.random.seed(seed=seed)

min_c_2, min_regret_2 = find_min_regret(np.linspace(0.05, 0.2, 5))

1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impressions have been simulated
70001 impressions have been simulated
80001 impressions have been simulated
90001 impressions have been simulated
100001 impressions have been simulated
110001 impressions have been simulated
120001 impressions have been simulated
130001 impressions have been simulated
140001 impressions have been simulated
150001 impressions have been simulated
160001 impressions have been simulated
170001 impressions have been simulated
180001 impressions have been simulated
190001 impressions have been simulated
c = 0.05
Time: 664.6530134677887

New results: 249.76800905727052, 0.0012488400452863526,  184

1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been 

In [10]:
seed = 18475
np.random.seed(seed=seed)

min_c_2, min_regret_2 = find_min_regret(np.linspace(0.06, 0.09, 5))

1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impressions have been simulated
70001 impressions have been simulated
80001 impressions have been simulated
90001 impressions have been simulated
100001 impressions have been simulated
110001 impressions have been simulated
120001 impressions have been simulated
130001 impressions have been simulated
140001 impressions have been simulated
150001 impressions have been simulated
160001 impressions have been simulated
170001 impressions have been simulated
180001 impressions have been simulated
190001 impressions have been simulated
c = 0.06
Time: 446.9205255508423

New results: 95.63600885613073, 0.00047818004428065367,  184

1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been 

Итого минимальный **regret = 95.63600885613073** при **c=0.06**.